In [1]:
:opt no-lint

In [2]:
import Test.Hspec
import Test.QuickCheck

# Chapter 14. Testing
## 14.1 Testing
* principle of testing.
* testing libraries `hspec` and `QuickCheck`.

## 14.2 A quick tour of testing for the uninitiated
Categories:
* unit testing (spec testing is a variant)
* property testing

## 14.3 Conventional testing
`hspec` is based on monadic actions. 
Write a function that multiplies two numbers using recursive summation. The type should be `(Eq a, Num a) => a -> a -> a`, and then write `hspec` tests for it.

## 14.4 Enter QuickCheck
`QuickCheck` was the first library to offer property testing. It requires concrete types in properties. The number of tests runs defaults to 100.
### Arbitrary instances
`QuickCheck` contains:
* a type class called `Arbitrary`. It is unprincipled because it has no laws.
nothing specific it’s supposed to do
* a newtype called `Gen` for generating its random data.

`arbitrary` is a value of type `Gen`. `sample` uses it to output random data.
`IO` allow to use a global resource of random values to generate the data.
We can use `QuickCheck` without `hspec` by means of `prop_additionGreater`.
### Intermission: Short exercise
Write `hspec` tests for recursive multiplications function of Chapter 8

In [33]:
multRecur :: Integral a => a -> a -> a
multRecur a 1 = a
multRecur a b = a + multRecur a (b - 1)

hspec $
  describe "multRecur" $ do
    it "multiplies by 0 correctly" $ multRecur 42 0 `shouldBe` 0
    --it "multiplies 1 correctly" $ multRecur 42 1 `shouldBe` 42
    --it "multiplies 2 by 2 correctly" $ multRecur 2 2 `shouldBe` 4
    --it "multiplies 22 by 84" $ multRecur 22 84 `shouldBe` 1848
    --it "multiplies -22 by 84" $ multRecur -22 84 `shouldBe` -1848

: 

## 14.5 Morse code
## 14.6Arbitrary instances
One of the more important parts of becoming an expert user of `QuickCheck` is learning to write instances of the `Arbitrary` type class for your datatypes.

## 14.7 Chapter exercises
Fill in the test cases that print question marks. If you think of additional tests you could perform, add them

In [27]:
import Data.List (intersperse, intercalate, map)

digitToWord :: Int -> String
digitToWord n =
  case n of
    0 -> "zero"
    1 -> "one"
    2 -> "two"
    3 -> "three"
    4 -> "four"
    5 -> "five"
    6 -> "six"
    7 -> "seven"
    8 -> "eight"
    9 -> "nine"
    _ -> "zero"

digits :: Int -> [Int]
digits n
  | n < 10    = [ mod n 10 ]
  | otherwise = concat [ digits $ div n 10, [ mod n 10 ] ]

wordNumber :: Int -> String
wordNumber = intercalate "-" . map digitToWord . digits

hspec $ do
  describe "digitToWord" $ do
    it "returns zero for 0" $ do
      digitToWord 0 `shouldBe` "zero"
    it "returns one for 1" $ do
      digitToWord 1 `shouldBe` "one"
  describe "digits" $ do
    it "returns [1] for 1" $ do
      digits 1 `shouldBe` [1]
    it "returns [1, 0, 0] for 100" $ do
      digits 100 `shouldBe` [1, 0, 0]
  describe "wordNumber" $ do
    it "one-zero-zero given 100" $ do
      wordNumber 100 `shouldBe` "one-zero-zero"
    it "nine-zero-zero-one for 9001" $ do
      wordNumber 9001 `shouldBe` "nine-zero-zero-one"

: 

### Using QuickCheck
Test some basic properties using QuickCheck:
1.

In [5]:
-- for a function
half x = x / 2
-- this property should hold
halfIdentity = (*2) . half

prop_halfIdentity :: (Fractional a, Eq a) => a -> Bool
prop_halfIdentity x = halfIdentity x == x

quickCheck prop_halfIdentity

+++ OK, passed 100 tests.

2.

In [6]:
import Data.List (sort)

-- for any list you apply sort to,
-- this property should hold
listOrdered :: (Ord a) => [a] -> Bool
listOrdered xs = snd $ foldr go (Nothing, True) xs
  where go _ status@(_, False) = status
        go y (Nothing, t) = (Just y, t)
        go y (Just x, t) = (Just y, x >= y)
    
quickCheck listOrdered

+++ OK, passed 100 tests.

3. Now, we’ll test the associative and commutative properties of addition. Keep in mind, these properties won’t hold for types based on IEEE-754 floating point numbers, such as `Float` or `Double`.

In [7]:
plusAssociative x y z = x + (y + z) == (x + y) + z

plusCommutative x y = x + y == y + x

quickCheck plusAssociative
quickCheck plusCommutative

+++ OK, passed 100 tests.

+++ OK, passed 100 tests.

4. Now do the same for multiplication.

In [8]:
plusAssociative x y z = x * (y * z) == (x * y) * z

plusCommutative x y = x * y == y * x

quickCheck plusAssociative
quickCheck plusCommutative

+++ OK, passed 100 tests.

+++ OK, passed 100 tests.

5. We mentioned in one of the first chapters that there are some laws involving the relationships of `quot` to `rem` and `div` to `mod`.
Write `QuickCheck` tests to prove them:

In [9]:
prop_quotRem :: (Integral a) => a -> Positive a -> Bool
prop_quotRem x (Positive y) = (quot x y) * y + (rem x y) == x

prop_divMod :: (Integral a) => a -> Positive a -> Bool
prop_divMod x (Positive y) = (div x y) * y + (mod x y) == x

quickCheck prop_quotRem
quickCheck prop_divMod

+++ OK, passed 100 tests.

+++ OK, passed 100 tests.

6. Is the `^` operation associative? Is it commutative? Use `QuickCheck` to see if the computer can contradict such an assertion.

In [10]:
prop_commutative x y = x ^ y == y ^ x

quickCheck prop_commutative

*** Failed! Falsified (after 2 tests and 3 shrinks):
0
1

7. Test that reversing a list twice is the same as the identity of the original list:

In [28]:
prop_reverseTwiceId :: [Int] -> Bool
prop_reverseTwiceId x = (reverse . reverse) x == x

quickCheck prop_reverseTwiceId

+++ OK, passed 100 tests.

8. Write a property for the definition of `$`:

In [29]:
import Test.QuickCheck.Function

prop_dollarApply :: (Eq a) => Fun a a -> a -> Bool
prop_dollarApply (Fn f) a = (f $ a) == f a

prop_dollarCompose :: (Eq a) => Fun a a -> Fun a a -> a -> Bool
prop_dollarCompose (Fn f) (Fn g) x = (f . g) x == (\x' -> f (g x')) x

quickCheck prop_dollarApply
quickCheck prop_dollarCompose

+++ OK, passed 100 tests.

+++ OK, passed 100 tests.

9. See if these two functions are equal:

In [30]:
prop_foldConsEqualsAppend xs ys = foldr (:) xs ys == (++) xs ys
prop_foldAppendEqualsConcat xs = foldr (++) [] xs == concat xs

quickCheck prop_foldConsEqualsAppend
quickCheck prop_foldAppendEqualsConcat

+++ OK, passed 100 tests.

+++ OK, passed 100 tests.

10. Hmm. Is that so?

In [14]:
prop_lengthTake f n xs = length (take n xs) == n
quickCheck prop_lengthTake

*** Failed! Falsified (after 2 tests):
()
1
[]

11. Finally, this is a fun one. You may remember we had you compose read and show one time to complete a “round trip.” Well, now you can test that it works:

In [15]:
prop_readShow f x = read (show x) == x
quickCheck prop_readShow

+++ OK, passed 100 tests.

### Failure
Find out why this property fails:

In [16]:
-- for a function
square x = x * x
-- Why does this property not hold?
-- Examine the type of sqrt.
squareIdentity = square . sqrt

prop_squareIdentity x = (square . sqrt) x == x
quickCheck prop_squareIdentity

*** Failed! Falsified (after 3 tests and 3 shrinks):
-1.0e-2

### Idempotence
Use QuickCheck and the following helper functions to demonstrate idempotence for the following:

In [17]:
twice f = f . f
fourTimes = twice . twice

1.

In [18]:
import Data.Char

capitalizeWord :: String -> String
capitalizeWord = fmap toUpper

prop_idempotence x = (capitalizeWord x == twice capitalizeWord x) 
  && (capitalizeWord x == fourTimes capitalizeWord x)
quickCheck prop_idempotence

+++ OK, passed 100 tests.

2.

In [19]:
prop_idempotence x = (sort x == twice sort x) && (sort x == fourTimes sort x)
quickCheck prop_idempotence

+++ OK, passed 100 tests.

### Make a Gen random generator for the datatype
Make `Gen` generators for different datatypes:
1. Equal probabilities for each:

In [20]:
data Fool = Fulse | Frue deriving (Eq, Show)

fooler :: Gen Fool
fooler = oneof [return Fulse, return Frue]
  
sample fooler

Fulse
Fulse
Fulse
Frue
Frue
Fulse
Frue
Frue
Frue
Fulse
Frue

2. 2/3s chance of Fulse, 1/3 chance of Frue:

In [21]:
data Fool = Fulse | Frue deriving (Eq, Show)

fooler :: Gen Fool
fooler = frequency [(2, return Fulse), (1, return Frue)]
  
sample fooler

Frue
Fulse
Fulse
Frue
Fulse
Fulse
Frue
Fulse
Frue
Fulse
Fulse

### Hangman testing

In [22]:
data Puzzle = Puzzle String [Maybe Char] [Char]

charInWord :: Puzzle -> Char -> Bool
charInWord (Puzzle word _ _) c = elem c word

alreadyGuessed :: Puzzle -> Char -> Bool
alreadyGuessed (Puzzle _ _ guessed) c = elem c guessed

fillInCharacter :: Puzzle -> Char -> Puzzle
fillInCharacter (Puzzle word filledInSoFar s) c = Puzzle word newFilledInSoFar (c : s)
  where zipper guessed wordChar guessChar = 
          if wordChar == guessed 
          then Just wordChar 
          else guessChar
        newFilledInSoFar = 
          let zd = zipper c
          in zipWith zd word filledInSoFar

handleGuess :: Puzzle -> Char -> IO Puzzle
handleGuess puzzle guess = do
  putStrLn ("Your guess was: " ++ [guess])
  return puzzle
  case (charInWord puzzle guess, alreadyGuessed puzzle guess) of
    (_, True) -> do
      putStrLn "You already guessed that, pick something else!"
      return puzzle
    (True, _) -> do
      putStrLn "Great guess! That's in, filling it accordingly."
      return (fillInCharacter puzzle guess)
    (False, _) -> do
      putStrLn "Too bad! This isn't in the word, try again."
      return (fillInCharacter puzzle guess)

In [31]:
hspec $ do
  describe "fillInCharacter" $ do
    it "ignores bad guess" $ do
      let word = "puzzle"
      let puzzle = newPuzzle word
      let puzzle' = Puzzle word (map (const Nothing) word) ['x']
      fillInCharacter puzzle 'x' `shouldBe` puzzle'

    it "takes good guess" $ do
      let word = "puzzle"
      let puzzle = newPuzzle word
      let discovered = (Just 'p') : tail (map (const Nothing) word)
      let puzzle' = Puzzle word discovered ['p']
      fillInCharacter puzzle 'p' `shouldBe` puzzle'

  describe "handleGuess" $ do
    it "ignores bad guess" $ do
      let word = "puzzle"
      let puzzle = newPuzzle word
      let puzzle' = Puzzle word (map (const Nothing) word) ['x']
      result <- handleGuess puzzle 'x'
      result `shouldBe` puzzle'

    it "takes good guess" $ do
      let word = "puzzle"
      let puzzle = newPuzzle word
      let discovered = (Just 'p') : tail (map (const Nothing) word)
      let puzzle' = Puzzle word discovered ['p']
      result <- handleGuess puzzle 'p'
      result `shouldBe` puzzle'

: 

### Validating ciphers
As a final exercise, create QuickCheck properties that verify that your Caesar and Vigenère ciphers return the same data after encoding and decoding a string.

In [24]:
import Data.Char (chr, ord, toUpper)


type PlainText = String
type CipherText = String


vigenère :: String -> PlainText -> CipherText
vigenère secret = zipWith (shift (+)) (cycle secret) . concat . words . capitalize

unvigenère :: String -> CipherText -> PlainText
unvigenère secret = zipWith (shift (-)) (cycle secret) . concat . words . capitalize

capitalize = fmap toUpper

shift :: (Int -> Int -> Int) -> Char -> Char -> Char
shift op offset ch = numToChar $ charToNum ch `op` charToNum offset
  where
    charToNum ch = ord ch - ord 'A'
    numToChar n = chr $ (n `mod` 26) + ord 'A'
    

cæsar :: Int -> PlainText -> CipherText
cæsar secret = map
  (((chr . (\x -> if x >= lastOrd then x - 26 else x)) . (+secret)) . ord)
  
uncæsar :: Int -> CipherText -> PlainText
uncæsar secret = map
  (((chr . (\x -> if x < firstOrd then x + 26 else x)) . (\x -> x - secret)) . ord)

firstOrd = ord 'a'
lastOrd = firstOrd + 26


genLetter :: Gen Char
genLetter = elements ['a' .. 'z']

genText :: Gen String
genText = listOf genLetter


quickCheck $
  forAll (arbitrary :: Gen Int) $ \ k ->
  forAll genText (\x -> (cæsar k . uncæsar k) x ==  x)
quickCheck $
  forAll (arbitrary :: Gen Int) $ \ k ->
  forAll genText (\x -> (uncæsar k . cæsar k) x ==  x)
quickCheck $
  forAll (genLetters `suchThat` (not . null)) $ \ k ->
  forAll genText (\x -> (capitalize . vigenère k . unvigenère k) x == capitalize x)
quickCheck $
  forAll (genLetters `suchThat` (not . null)) $ \ k ->
  forAll genText (\x -> (capitalize . unvigenère k . vigenère k) x == capitalize x)

+++ OK, passed 100 tests.

+++ OK, passed 100 tests.

: 